# 🧮 Bioeconomic Linear Program (MVP)
This notebook builds a single-year bioeconomic linear program with:
- Households, crops, livestock
- Land, labor, and calorie constraints
- Scenario multipliers (yields, prices, wages, fertilizer, population)

It uses `scipy.optimize.linprog` (no external LP solvers). Edit the CSV templates it creates, then re-run.

In [2]:
import json
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.optimize import linprog
# from caas_jupyter_tools import display_dataframe_to_user

print("Loaded: numpy, pandas, scipy.optimize.linprog")

Loaded: numpy, pandas, scipy.optimize.linprog


In [ ]:
@dataclass
class HouseholdClass:
    name: str
    n_households: float
    adult_equiv: float
    labor_endowment: float
    land_available: float
    max_hired_labor: float  # cap to avoid unbounded labor arbitrage

@dataclass
class CropParam:
    name: str
    calorie_per_kg: float
    yield_per_ha: float
    price_sale: float
    seed_cost_per_ha: float
    fert_cost_per_ha: float
    chem_cost_per_ha: float
    labor_req_per_ha: float

@dataclass
class LivestockParam:
    name: str
    price_sale: float
    feed_cost_per_unit: float
    vet_cost_per_unit: float
    labor_req_per_unit: float

@dataclass
class PriceParam:
    wage: float

@dataclass
class ScenarioShocks:
    yield_multiplier: float = 1.0
    crop_price_multiplier: float = 1.0
    wage_multiplier: float = 1.0
    fert_price_multiplier: float = 1.0
    population_multiplier: float = 1.0

@dataclass
class ModelParams:
    households: Dict[str, HouseholdClass]
    crops: Dict[str, CropParam]
    livestock: Dict[str, LivestockParam]
    prices: PriceParam
    min_kcal_per_person_per_day: float = 2000.0
    days_per_year: int = 365

print("Schemas defined.")

Schemas defined.


In [4]:
TEMPLATE_DIR = Path("./data")  # relative to notebook
TEMPLATE_DIR.mkdir(parents=True, exist_ok=True)

# Households template (add all 6 classes later)
households_df = pd.DataFrame([
    {"name": "HFR", "n_households": 100, "adult_equiv": 3.8, "labor_endowment": 220, "land_available": 0.5, "max_hired_labor": 120},
    {"name": "MMIR", "n_households": 120, "adult_equiv": 4.6, "labor_endowment": 320, "land_available": 1.1, "max_hired_labor": 150},
])
households_df.to_csv(TEMPLATE_DIR / "households.csv", index=False)

# Crops template
crops_df = pd.DataFrame([
    {"name": "maize", "calorie_per_kg": 3600, "yield_per_ha": 2000, "price_sale": 0.4,
     "seed_cost_per_ha": 35, "fert_cost_per_ha": 90, "chem_cost_per_ha": 25, "labor_req_per_ha": 55},
    {"name": "beans", "calorie_per_kg": 3400, "yield_per_ha": 1200, "price_sale": 0.7,
     "seed_cost_per_ha": 40, "fert_cost_per_ha": 50, "chem_cost_per_ha": 20, "labor_req_per_ha": 65},
])
crops_df.to_csv(TEMPLATE_DIR / "crops.csv", index=False)

# Livestock template
livestock_df = pd.DataFrame([
    {"name": "goat", "price_sale": 50, "feed_cost_per_unit": 10, "vet_cost_per_unit": 5, "labor_req_per_unit": 8},
    {"name": "chicken", "price_sale": 6, "feed_cost_per_unit": 1.2, "vet_cost_per_unit": 0.4, "labor_req_per_unit": 0.8},
])
livestock_df.to_csv(TEMPLATE_DIR / "livestock.csv", index=False)

# Prices template
prices_df = pd.DataFrame([{"wage": 2.5}])
prices_df.to_csv(TEMPLATE_DIR / "prices.csv", index=False)

# Scenario template
scenario = {
    "yield_multiplier": 1.0,
    "crop_price_multiplier": 1.0,
    "wage_multiplier": 1.0,
    "fert_price_multiplier": 1.0,
    "population_multiplier": 1.0,
}
with open(TEMPLATE_DIR / "scenario.json", "w") as f:
    json.dump(scenario, f, indent=2)

# Display templates for quick review (if supported)
try:
    display_dataframe_to_user("households.csv", households_df)
    display_dataframe_to_user("crops.csv", crops_df)
    display_dataframe_to_user("livestock.csv", livestock_df)
    display_dataframe_to_user("prices.csv", prices_df)
except Exception:
    pass

print("Templates written to ./data")


Templates written to ./data


In [5]:
def load_params_from_folder(folder: Path) -> Tuple[ModelParams, ScenarioShocks]:
    hh = pd.read_csv(folder / "households.csv")
    crops = pd.read_csv(folder / "crops.csv")
    livest = pd.read_csv(folder / "livestock.csv")
    prices_df = pd.read_csv(folder / "prices.csv")
    with open(folder / "scenario.json", "r") as f:
        sc_dict = json.load(f)

    households = {
        r["name"]: HouseholdClass(
            name=r["name"],
            n_households=float(r["n_households"]),
            adult_equiv=float(r["adult_equiv"]),
            labor_endowment=float(r["labor_endowment"]),
            land_available=float(r["land_available"]),
            max_hired_labor=float(r.get("max_hired_labor", 0.0))
        )
        for _, r in hh.iterrows()
    }
    crops_map = {
        r["name"]: CropParam(
            name=r["name"],
            calorie_per_kg=float(r["calorie_per_kg"]),
            yield_per_ha=float(r["yield_per_ha"]),
            price_sale=float(r["price_sale"]),
            seed_cost_per_ha=float(r["seed_cost_per_ha"]),
            fert_cost_per_ha=float(r["fert_cost_per_ha"]),
            chem_cost_per_ha=float(r["chem_cost_per_ha"]),
            labor_req_per_ha=float(r["labor_req_per_ha"]),
        )
        for _, r in crops.iterrows()
    }
    livest_map = {
        r["name"]: LivestockParam(
            name=r["name"],
            price_sale=float(r["price_sale"]),
            feed_cost_per_unit=float(r["feed_cost_per_unit"]),
            vet_cost_per_unit=float(r["vet_cost_per_unit"]),
            labor_req_per_unit=float(r["labor_req_per_unit"]),
        )
        for _, r in livest.iterrows()
    }
    prices = PriceParam(wage=float(prices_df.iloc[0]["wage"]))
    scenario = ScenarioShocks(**sc_dict)
    params = ModelParams(households=households, crops=crops_map, livestock=livest_map, prices=prices)
    return params, scenario

print("Loaders ready.")


Loaders ready.


In [6]:
def build_index_maps(H, C, L):
    idx = {}
    pos = 0
    for h in H:
        for c in C:
            idx[("area", h, c)] = pos; pos += 1
    for h in H:
        for c in C:
            idx[("cons", h, c)] = pos; pos += 1
    for h in H:
        for c in C:
            idx[("sold", h, c)] = pos; pos += 1
    for h in H:
        for c in C:
            idx[("stored", h, c)] = pos; pos += 1
    for h in H:
        idx[("hired", h, None)] = pos; pos += 1
    for h in H:
        idx[("off_farm", h, None)] = pos; pos += 1
    for h in H:
        for l in L:
            idx[("live_units", h, l)] = pos; pos += 1
    return idx, pos

print("Indexing helpers ready.")


Indexing helpers ready.


In [ ]:
def solve_single_year_lp(params: ModelParams, scenario: ScenarioShocks, hh_subset: Optional[List[str]] = None, calories_floor_override: Optional[float] = None):
    H = hh_subset if hh_subset is not None else list(params.households.keys())
    C = list(params.crops.keys())
    L = list(params.livestock.keys())

    idx, nvars = build_index_maps(H, C, L)
    c = np.zeros(nvars)

    ym = scenario.yield_multiplier
    pm = scenario.crop_price_multiplier
    wm = scenario.wage_multiplier
    fm = scenario.fert_price_multiplier

    # Objective (maximize net => minimize negative)
    for h in H:
        c[idx[("hired", h, None)]] += (params.prices.wage * wm)     # hired labor cost
        c[idx[("off_farm", h, None)]] += -(params.prices.wage * wm) # off-farm wage income
        for cn in C:
            cp = params.crops[cn]
            c[idx[("sold", h, cn)]] += -(cp.price_sale * pm)         # crop sales revenue
            per_ha_cost = cp.seed_cost_per_ha + fm * cp.fert_cost_per_ha + cp.chem_cost_per_ha
            c[idx[("area", h, cn)]] += per_ha_cost                   # per-ha input costs
        for l in L:
            lv = params.livestock[l]
            c[idx[("live_units", h, l)]] += -(lv.price_sale)         # livestock revenue
            c[idx[("live_units", h, l)]] += (lv.feed_cost_per_unit + lv.vet_cost_per_unit)  # livestock costs

    A_eq, b_eq, A_ub, b_ub = [], [], [], []
    bounds = [(0, None) for _ in range(nvars)]

    # Production balance: yield*area - cons - sold - stored = 0
    for h in H:
        for cn in C:
            row = np.zeros(nvars)
            row[idx[("area", h, cn)]] = params.crops[cn].yield_per_ha * ym
            row[idx[("cons", h, cn)]] = -1.0
            row[idx[("sold", h, cn)]] = -1.0
            row[idx[("stored", h, cn)]] = -1.0
            A_eq.append(row); b_eq.append(0.0)

    # Land: sum area <= land_available
    for h in H:
        row = np.zeros(nvars)
        for cn in C: row[idx[("area", h, cn)]] = 1.0
        A_ub.append(row); b_ub.append(params.households[h].land_available)

    # Labor: crop + livestock + off_farm <= family + hired
    for h in H:
        row = np.zeros(nvars)
        for cn in C: row[idx[("area", h, cn)]] = params.crops[cn].labor_req_per_ha
        for l in L: row[idx[("live_units", h, l)]] = params.livestock[l].labor_req_per_unit
        row[idx[("off_farm", h, None)]] = 1.0
        row[idx[("hired", h, None)]] = -1.0
        A_ub.append(row); b_ub.append(params.households[h].labor_endowment)

    # Hired labor cap: hired <= max_hired_labor
    for h in H:
        row = np.zeros(nvars)
        row[idx[("hired", h, None)]] = 1.0
        A_ub.append(row); b_ub.append(params.households[h].max_hired_labor)

    # Calories: -sum(cons*cal) <= -need  (i.e., sum(cons*cal) >= need)
    for h in H:
        row = np.zeros(nvars)
        for cn in C: row[idx[("cons", h, cn)]] = -params.crops[cn].calorie_per_kg
        kcal_floor = (calories_floor_override if calories_floor_override is not None else params.min_kcal_per_person_per_day)
        kcal_need = kcal_floor * (params.households[h].adult_equiv * scenario.population_multiplier) * params.days_per_year
        A_ub.append(row); b_ub.append(-kcal_need)

    res = linprog(c,
                  A_ub=np.array(A_ub) if A_ub else None, b_ub=np.array(b_ub) if b_ub else None,
                  A_eq=np.array(A_eq) if A_eq else None, b_eq=np.array(b_eq) if b_eq else None,
                  bounds=bounds, method="highs")

    out = {
        "success": bool(res.success),
        "status": res.message,
        "objective": None,
        "by_household": {}
    }
    if not res.success:
        return out

    out["objective"] = -res.fun
    x = res.x

    def val(kind, h, k=None):
        return float(x[idx[(kind, h, k)]]) if (kind, h, k) in idx else 0.0

    for h in H:
        land_used = sum(val("area", h, cn) for cn in C)
        hired = val("hired", h)
        off = val("off_farm", h)
        kcal_from_crops = sum(val("cons", h, cn) * params.crops[cn].calorie_per_kg for cn in C)
        kcal_need = params.min_kcal_per_person_per_day * params.households[h].adult_equiv * params.days_per_year

        rev_crops = sum(val("sold", h, cn) * (params.crops[cn].price_sale * scenario.crop_price_multiplier) for cn in C)
        cost_crop_inputs = sum(val("area", h, cn) * (params.crops[cn].seed_cost_per_ha + scenario.fert_price_multiplier * params.crops[cn].fert_cost_per_ha + params.crops[cn].chem_cost_per_ha) for cn in C)
        rev_livestock = sum(val("live_units", h, l) * params.livestock[l].price_sale for l in L)
        cost_livestock = sum(val("live_units", h, l) * (params.livestock[l].feed_cost_per_unit + params.livestock[l].vet_cost_per_unit) for l in L)
        rev_off = off * (params.prices.wage * scenario.wage_multiplier)
        cost_hired = hired * (params.prices.wage * scenario.wage_multiplier)

        crop_rows = []
        for cn in C:
            prod = val("area", h, cn) * (params.crops[cn].yield_per_ha * scenario.yield_multiplier)
            crop_rows.append({
                "crop": cn,
                "area_ha": val("area", h, cn),
                "production_kg": prod,
                "sold_kg": val("sold", h, cn),
                "consumed_kg": val("cons", h, cn),
                "stored_kg": val("stored", h, cn),
            })
        live_rows = []
        for l in L:
            live_rows.append({"livestock": l, "units": val("live_units", h, l)})

        out["by_household"][h] = {
            "land_used": land_used,
            "hired_labor": hired,
            "off_farm": off,
            "kcal_from_crops": kcal_from_crops,
            "kcal_need": kcal_need,
            "revenue_breakdown": {
                "crops": rev_crops,
                "livestock": rev_livestock,
                "off_farm_wage": rev_off,
            },
            "cost_breakdown": {
                "crop_inputs": cost_crop_inputs,
                "livestock_costs": cost_livestock,
                "hired_labor_cost": cost_hired,
            },
            "crop_details": crop_rows,
            "livestock_details": live_rows,
        }

    return out

print("Solver ready.")

Solver ready.


In [ ]:
# Load from ./data and run a demo solve
params, scenario = load_params_from_folder(Path("./data"))
demo_results = solve_single_year_lp(params, scenario, hh_subset=None)

print("Status:", demo_results["status"])
print("Success:", demo_results["success"])
print("Objective (net income):", demo_results["objective"])

# Pretty report
if demo_results["success"]:
    rows = []
    for h, hd in demo_results["by_household"].items():
        rows.append({
            "household_class": h,
            "land_used_ha": hd["land_used"],
            "hired_labor_days": hd["hired_labor"],
            "off_farm_days": hd["off_farm"],
            "kcal_from_crops": hd["kcal_from_crops"],
            "kcal_need": hd["kcal_need"],
            "rev_crops": hd["revenue_breakdown"]["crops"],
            "rev_livestock": hd["revenue_breakdown"]["livestock"],
            "rev_off_farm": hd["revenue_breakdown"]["off_farm_wage"],
            "cost_crop_inputs": hd["cost_breakdown"]["crop_inputs"],
            "cost_livestock": hd["cost_breakdown"]["livestock_costs"],
            "cost_hired_labor": hd["cost_breakdown"]["hired_labor_cost"],
        })
    summary_df = pd.DataFrame(rows)
    try:
        display_dataframe_to_user("Model Summary (edit ./data/*.csv then re-run)", summary_df)
    except Exception:
        display(summary_df)

# Save JSON
Path("./outputs").mkdir(parents=True, exist_ok=True)
with open("./outputs/demo_results.json", "w") as f:
    json.dump(demo_results, f, indent=2)

print("Saved results to ./outputs/demo_results.json")

Status: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Success: True
Objective (net income): 3673.375


,household_class,land_used_ha,hired_labor_days,off_farm_days,kcal_from_crops,kcal_need,rev_crops,rev_livestock,rev_off_farm,cost_crop_inputs,cost_livestock,cost_hired_labor
0,HFR,0.5,120.0,0.0,2774000.0,2774000.0,96.366667,2335.145833,0.0,70.411111,622.705556,300.0
1,MMIR,1.1,150.0,0.0,3358000.0,3358000.0,532.233333,3023.729167,0.0,139.655556,806.327778,375.0


Saved results to ./outputs/demo_results.json
